In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import statistics
from scipy.stats import norm, t
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab
import scipy


In [7]:
#Annual VaR and CVar Engine Utilizng the Variance and Covariance Method
list_assets = ["HSBC", "BCS", "TSLA", "NVDA"]
dict_assets = {}
initial_portfolio = 100000
time = 250

def hour_wiper(df):
    
    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df = df.set_index('Date')
    
    return df

def data_grabber(tickers, assets):
    #Function designed to take any ticker and grab annual data
    for x in tickers:
        try: 
            
            ticker = yf.Ticker(x)
            ticker_data = ticker.history(period = "1y", interval = "1d")
            ticker_data = hour_wiper(ticker_data)
            if not ticker_data.empty:
                assets[x] = ticker_data
            else:
                print(f"The associated data for {x} is empty")
        except Exception as e:
            print(f"There was an error in fetching {x} data")
            
    return assets

def returns(dict_assets):
    #Calculates daily returns of each asset in the portfolio and puts them into a seperate dataframe
    for x in dict_assets:
        df_common = dict_assets[x]    
        df_common["Returns"] = df_common["Close"].pct_change()
        df_common = df_common.dropna()        
        dict_assets[x] = df_common
    
    dict_returns = {}    
    for x in dict_assets:
        df_common = dict_assets[x]
        series_returns = df_common["Returns"]
        dict_returns[x] = series_returns
        
    df_returns = pd.concat(dict_returns.values(), axis = 1, keys=dict_returns)
    mean_returns = df_returns.mean()
    
    return dict_assets, df_returns, mean_returns

def weights(dict_assets, list_weights):
    #Takes weights and inserts them into dataframes of each asset as a column 
    
    list_counter = 0
    if isinstance(list_weights, list):
        for x in list_weights:
            x = float(x)
            x = x/100
            list_weights[list_counter] = x
            list_counter += 1
        array_weights = np.array(list_weights)
    else:
        raise TypeError("the expected data structure for the weights is a list")
    
    if np.sum(array_weights) == 1:
        pass
    else:
        print(f"The sum of your weights does not equal to 1")
    
    weights_counter = 0
    if  len(array_weights) == len(dict_assets):
        for key in dict_assets:
            df_common = dict_assets[key]
            df_common["Weights"] = list_weights[weights_counter]
            df_common = df_common.dropna()
            dict_assets[key] = df_common
            weights_counter += 1
    else:
        print(f"The number of assets and weights do not match up, please try again")
    
    return dict_assets, array_weights

def portfolio(dict_assets):
    #Assembles the portfolio into a singular dataframe
    df_portfolio = pd.concat(dict_assets.values(), axis = 1, keys=dict_assets.keys())
    df_portfolio = df_portfolio.reset_index()

    return df_portfolio
    
def portfolio_return(df_portfolio, df_returns, array_weights):
    #Calculates the daily returns of the portfolio given the weights
        
    series_portfolio_return = df_returns.dot(array_weights)
    df_portfolio["Portfolio Returns"] = series_portfolio_return
    
    return df_portfolio, df_returns, series_portfolio_return
    
def covariance_matrix(dict_assets, df_returns):
    #Finds the covariance between the returns of each asset in the portfolio        
    df_cov = df_returns.cov()
    
    return df_cov
  
def portfolio_variance_calculator(df_cov, array_weights):
    #Find portfolio variance given covariance matrix and weights of the portfolio
    array_portfolio_variance = np.dot(array_weights.T, np.dot(df_cov, array_weights))
    float_portfolio_variance = float(array_portfolio_variance)
    return float_portfolio_variance  
    
def portfolio_std(float_portfolio_variance):
    #Multiply by the square root of time to annualize the volatility
    float_portfolio_std = (math.sqrt(float_portfolio_variance))*np.sqrt(time)
    
    return float_portfolio_std
    
def portfolio_mean_return(mean_returns, array_weights):
    #Multiply by the number of trading days to annualize average portfolio return
    P_returns = np.sum(mean_returns*array_weights)*time

    
    return P_returns

def Var_Parametric(portfolio_return, std, distribution, dof = 6):
    """Calculate the portfolio VaR Given a distribution with known parameters"""
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_VaR = {}
    
    for x in list_alpha:
        if distribution == "normal":
            VaR = round((norm.ppf(1-x) * std - portfolio_return)*initial_portfolio, 2)*-1
        elif distribution == "t-distribution":
            nu = dof
            VaR = round((np.sqrt((nu-2)/nu) * t.ppf(1-x, nu)* std - portfolio_return * initial_portfolio, 2))*-1
        else:
            raise TypeError("Expected distribution to be normal or t-distribution")
        dict_VaR[x] = VaR
    print(f"Annual Var with associated confidence levels: {dict_VaR}")    
        
    return dict_VaR

def CVar_Parametric(portfolio_return, std, distribution = "normal", dof = 6):
    """Calculate the portfolio CVaR Given a distribution with known parameters"""
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_CVaR = {}
    
    for x in list_alpha:
        if distribution == "normal":
            CVaR = round(((1-x)**-1 * norm.pdf(norm.ppf(1-x))* std - portfolio_return)*initial_portfolio,2)*-1
        elif distribution == "t-distribution":
            nu = dof
            x_anu = t.ppf(1-x, nu)
            CVaR = round((-1/(1-x) * (1-nu)**-1 * (nu-2+x_anu**2) * t.ppf(x_anu, nu)* std - portfolio_return)*initial_portfolio, 2)*-1
        else:
            raise TypeError("Expected distribution to be normal or t-distribution")
        dict_CVaR[x] = CVaR
    print(f"Annual CVar with associated confidence levels: {dict_CVaR}")    
        
    return dict_CVaR

In [8]:
list_weights = ["20", "30", "40", "10"]

In [9]:
data_grabber(list_assets, dict_assets)
dict_assets, df_returns, mean_returns = returns(dict_assets)
df_portfolio = portfolio(dict_assets)
dict_assets, array_weights = weights(dict_assets, list_weights)
df_portfolio, df_returns, series_portfolio_returns = portfolio_return(df_portfolio, df_returns, array_weights)
df_cov = covariance_matrix(dict_assets, df_returns)
portfolio_variance = portfolio_variance_calculator(df_cov, array_weights)
std = portfolio_std(portfolio_variance)
Pret = portfolio_mean_return(mean_returns, array_weights)
Var_Parametric(Pret, std, "normal")
CVar_Parametric(Pret, std, "normal")

Annual Var with associated confidence levels: {0.9: 55520.33, 0.95: 66019.72, 0.975: 75126.37, 0.99: 85714.82}
Annual CVar with associated confidence levels: {0.9: -32235.16, 0.95: -41128.51, 0.975: -49078.58, 0.99: -58540.73}


{0.9: -32235.16, 0.95: -41128.51, 0.975: -49078.58, 0.99: -58540.73}